In [1]:
import cv2
from ultralytics import YOLO
import numpy as np

### Initialising a pre-trained YOLO model to detect people

In [2]:
person_model = YOLO("yolov8m.pt")

# persons = person_model("imagess.jpg", classes=[0])


image 1/1 C:\Users\User\Documents\DS and ML\Bed-availability-Detection\imagess.jpg: 480x640 6 persons, 200.7ms
Speed: 2.0ms preprocess, 200.7ms inference, 1.3ms postprocess per image at shape (1, 3, 480, 640)


### Loading our previously trained Bed Detection model

In [3]:
bed_model = YOLO("D:/runs/detect/train3/weights/best.pt")

##### Calculate the Intersection over Union (IoU) between two bounding boxes of two models

In [23]:
def iou(boxA, boxB):
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])

    interArea = max(0, xB - xA) * max(0, yB - yA) #intersection area

    boxAArea = (boxA[2] - boxA[0]) * (boxA[3] - boxA[1])
    boxBArea = (boxB[2] - boxB[0]) * (boxB[3] - boxB[1])
    unionArea = boxAArea + boxBArea - interArea #union area

    return interArea/float(unionArea) if unionArea > 0 else 0

##### Detect beds and people, then determine if a person is on a bed.

In [24]:
def detect_bed_occupancy(media_path):

    bed_results = bed_model(media_path, save=True)
    person_results = person_model(media_path, classes=[0])  
    
    bed_boxes = [list(map(int, box.xyxy[0].tolist())) for box in bed_results[0].boxes]
    person_boxes = [list(map(int, box.xyxy[0].tolist())) for box in person_results[0].boxes]

    img = cv2.imread(media_path)
    if img is None:
        print(f"Error: Could not load image : {media_path}")
        return 0
    img_arr = np.array(media_path)

    occupied_beds = set()
    assigned_beds = set()
    
    print(f"Total Beds Detected: {len(bed_boxes)}")
    print(f"Total Persons Detected: {len(person_boxes)}")

    for person in person_boxes:
        person_x1, person_y1, person_x2, person_y2 = person

        # Ignoring people standing
        person_width = person_x2 - person_x1
        person_height = person_y2 - person_y1
        if person_height > 1.8 * person_width:
            print(f"Person {person} ignored (Bro was standing)")
            continue

        best_bed_idx = None
        max_iou = 0

        for bed_idx, bed in enumerate(bed_boxes):
            if bed_idx in assigned_beds:  
                continue  # Skip if this bed is already assigned to another person

            overlap_score = iou(person, bed)  # Comparing the bounding boxes
            print(f"IOU Person {person} vs Bed {bed} = {overlap_score:.2f}")

            if overlap_score >= 0.08 and overlap_score > max_iou:  
                max_iou = overlap_score
                best_bed_idx = bed_idx

        if best_bed_idx is not None:
            occupied_beds.add(best_bed_idx)
            assigned_beds.add(best_bed_idx)
            print(f"Person {person} assigned to the Bed {bed_boxes[best_bed_idx]}")
            cv2.rectangle(img, 
                          (bed_boxes[best_bed_idx][0], bed_boxes[best_bed_idx][1]), 
                          (bed_boxes[best_bed_idx][2], bed_boxes[best_bed_idx][3]), 
                          (0, 0, 255), 2) # Draw red rectangle on occupied beds (don't know why this not working yet)

    # Draw all beds and people
    for box in bed_boxes:
        cv2.rectangle(img, (box[0], box[1]), (box[2], box[3]), (255, 0, 0), 2)  # Blue for beds
    for box in person_boxes:
        cv2.rectangle(img, (box[0], box[1]), (box[2], box[3]), (0, 255, 0), 2)  # Green for persons

    print(f"Occupied Beds: {len(occupied_beds)}")

    cv2.imshow("Bed Occupancy Detection", img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

    return len(occupied_beds)

### Testing the final product

In [25]:
detect_bed_occupancy("img2.jpg")


image 1/1 C:\Users\User\Documents\DS and ML\Bed-availability-Detection\img2.jpg: 640x640 2 hospital beds, 253.3ms
Speed: 0.0ms preprocess, 253.3ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)
Results saved to D:\runs\detect\predict5

image 1/1 C:\Users\User\Documents\DS and ML\Bed-availability-Detection\img2.jpg: 640x640 2 persons, 218.8ms
Speed: 4.3ms preprocess, 218.8ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)
Total Beds Detected: 2
Total Persons Detected: 2
IOU Person [364, 315, 446, 398] vs Bed [52, 386, 503, 704] = 0.01
IOU Person [364, 315, 446, 398] vs Bed [325, 317, 597, 508] = 0.13
Person [364, 315, 446, 398] assigned to the Bed [325, 317, 597, 508]
IOU Person [136, 322, 500, 565] vs Bed [52, 386, 503, 704] = 0.39
Person [136, 322, 500, 565] assigned to the Bed [52, 386, 503, 704]
Occupied Beds: 2


2